# Introduction

The [https://badssl.com](https://badssl.com) website offers a variety of unique scenarios with TLS protocols, cipher suites, and certificates.  In this case, we are using the web site to identify the difference between certificate thumbprints and thrumbprints against each certificate SubjectPublicKeyInfo field.  This work borrows from the research identified from the [OpenSSL Notebook](./openssl.ipynb).

## Computing Environemnt

Some basic information about the environment in which the Nobeook is executing.

In [ ]:
# What environment am I running in?
!echo "Kernel:" && uname -a && echo ""
!echo "MacOS Version:" && sw_vers && echo ""
!echo "OpenSSL Version:" && openssl version && echo ""
!echo "Python Version:" && python --version && echo ""

## Acquire Subdomains

Crawl the top-level page of [https://badssl.com](https://badssl.com) for any related sub-domains.

In [ ]:
%%bash
FOLDER="data/badssl"
DOMAIN_LIST="${FOLDER}/domains.txt"
mkdir -p "${FOLDER}"

curl -s https://badssl.com/ |\
    grep -Eo '<a href="(.*badssl\.com.*?)"' |\
    awk -F '/' '{print $3}' |\
    sort | uniq > "${DOMAIN_LIST}"

echo "Domains found: $(wc -l ${DOMAIN_LIST} | awk '{print $1}')"
echo ""
cat ${DOMAIN_LIST}


## Acquire Certificates by Domain

Using the domain target list, retireve the certificates presented for each domain.

> Note: Some of the subdomains offer risky TLS configurations which modern versions of `openssl` may not support.  You can expect some errors to happen due to this incompatibility.  There will be enough sample data to illustrate the differences between thumbprint types.

In [ ]:
%%bash
BASE_FOLDER="data/badssl"
CERT_FOLDER="${BASE_FOLDER}/certificates"
mkdir -p "${CERT_FOLDER}"
DOMAIN_LIST="${BASE_FOLDER}/domains.txt"

for DOMAIN in $(cat ${DOMAIN_LIST}); 
    do
    # assign port 443 if not specified
    PORT=$(echo "${DOMAIN}" | awk -F ':' '{ if (NF == 1) print 443; else if (NF == 2) print $2 }')
    DOMAIN=$(echo "${DOMAIN}" | awk -F ':' '{print $1}')

    echo "Acquiring Certificate: https://${DOMAIN}:${PORT}"

    DOMAIN=$(echo "${DOMAIN}" | awk -F ':' '{print $1}')
    
    echo "" | openssl s_client -showcerts -servername "${DOMAIN}" -connect "${DOMAIN}:${PORT}" 2>/dev/null |\
        openssl x509 -outform PEM -out "${CERT_FOLDER}/${DOMAIN}_${PORT}"_signed-public-key.pem;

    openssl x509 -inform PEM -in "${CERT_FOLDER}/${DOMAIN}_${PORT}"_signed-public-key.pem \
        -outform DER -out "${CERT_FOLDER}/${DOMAIN}_${PORT}"_signed-public-key.der
    done

## Display Basic Certificate Information

Displays select fields from each certificate.
- Subject
- Issuer
- Serial Number
- Validity Period

In [ ]:
%%bash
FOLDER="data/badssl/certificates"

echo "Basic Certificate Details:"
for CERTIFICATE_PUBLIC_KEY_PEM in "${FOLDER}"/*_signed-public-key.pem; do
    echo "--------------------------------------------"
    DOMAIN=$(basename "${CERTIFICATE_PUBLIC_KEY_PEM}" | awk -F '_' '{print $1}')
    PORT=$(basename "${CERTIFICATE_PUBLIC_KEY_PEM}" | awk -F '_' '{print $2}')
    echo "Domain: ${DOMAIN}:${PORT}"
    openssl x509 -noout -in "${CERTIFICATE_PUBLIC_KEY_PEM}" \
        -subject -issuer -serial -startdate -enddate
done
echo ""


## Extract SubjectPublicKeyInfo

Process each certificate and extract the SubjectPublicKeyInfo field from the certificate file.

In [ ]:
%%bash
FOLDER="data/badssl/certificates"

for CERTIFICATE_PEM in "${FOLDER}"/*_signed-public-key.pem; do
    # strip the path and suffix from the file name.
    TARGET=$(echo "${CERTIFICATE_PEM}" | sed 's/.*\///g' | sed -E 's/\_signed-public-key.(der|pem)$//g')
    
    PUBLIC_KEY_INFO_PEM="${FOLDER}/${TARGET}_public-key-info.pem"
    PUBLIC_KEY_INFO_DER="${FOLDER}/${TARGET}_public-key-info.der"
    echo "---"
    echo "Extracting SubjectPublicKeyInfo: ${CERTIFICATE_PEM}"

    echo "                     Saving PEM: ${PUBLIC_KEY_INFO_PEM}"
    openssl x509 -pubkey -inform PEM -in "${CERTIFICATE_PEM}" | \
        sed -n '/BEGIN\ PUBLIC\ KEY/,/END\ PUBLIC\ KEY/p' > "${PUBLIC_KEY_INFO_PEM}"

    echo "          Converting PEM to DER: ${PUBLIC_KEY_INFO_DER}"
    openssl asn1parse -inform PEM -in "${PUBLIC_KEY_INFO_PEM}" -out "${PUBLIC_KEY_INFO_DER}"
done
echo ""

## Count of Thumbprint Types

Basic metrics on different thumbprint types.  _Certificate thumbprints_ represents the message digest of the binary (DER) certificate file.  _SPKI thumbprints_ represnets the message digest of the certificate's SubjectPublicKeyInfo field.

In [ ]:
%%bash
FOLDER="data/badssl/certificates"

COUNT_CERTIFICATES=$(ls "${FOLDER}"/*signed-public-key.der | wc -l | awk '{print $1}')

UNIQUE_DER_DIGEST_COUNT=$(sha256sum "${FOLDER}"/*signed-public-key*.der \
    | awk '{print $1}' | sort | uniq | wc -l | awk '{print $1}')

UNIQUE_SPKI_DIGEST_COUNT=$(sha256sum "${FOLDER}"/*public-key-info.der \
    | awk '{print $1}' | sort | uniq | wc -l | awk '{print $1}')


echo "Certificate Summary:"
echo "  Total Certificates: ${COUNT_CERTIFICATES}"
echo "  Unique Thumbprints:"
echo "        Certificates: ${UNIQUE_DER_DIGEST_COUNT}"
echo "                SPKI: ${UNIQUE_SPKI_DIGEST_COUNT}"
echo ""

echo "Certificate (DER) Digests (SHA256):"
sha256sum "${FOLDER}"/*signed-public-key.der | sort
echo ""

echo "SubjectPublicKeyInfo Digests (SHA256):"
echo "[SubjectPublicKeyInfo derived from the Private Key]"
sha256sum "${FOLDER}"/*public-key-info.der | sort


In [ ]:
%%bash
BASE_FOLDER="data/badssl"
CERT_FOLDER="${BASE_FOLDER}/certificates"
CSV="${BASE_FOLDER}"/certificate_inventory.csv
D="\t" 

echo -e "Target${D}Subject${D}SerialNumber${D}StartDate${D}EndDate${D}Thumbprint (SHA256)${D}SPKI (SHA256)" > "${CSV}"

for CERTIFICATE_DER in  "${CERT_FOLDER}"/*_signed-public-key.der; do
    # strip the path and suffix from the file name.
    TARGET=$(echo "${CERTIFICATE_DER}" | sed 's/.*\///g' | sed -E 's/\_signed-public-key.(der|pem)$//g')
    SUBJECT=$(openssl x509 -subject -noout -inform DER -in "${CERTIFICATE_DER}" | awk -F 'subject=' '{print $2}')
    SERIAL_NUMBER=$(openssl x509 -serial -noout -inform DER -in "${CERTIFICATE_DER}" | awk -F '=' '{print $2}')
    START_DATE=$(openssl x509 -startdate -noout -inform DER -in "${CERTIFICATE_DER}" | awk -F '=' '{print $2}')
    END_DATE=$(openssl x509 -enddate -noout -inform DER -in "${CERTIFICATE_DER}" | awk -F '=' '{print $2}')
    CERT_THUMBPRINT=$(sha256sum "${CERTIFICATE_DER}" | awk -F ' ' '{print $1}')
    SPKI_THUMBPRINT=$(sha256sum "${CERT_FOLDER}"/"${TARGET}"_public-key-info.der | awk '{print $1}')

    echo "--------------------------------"
    echo "      Target: ${TARGET}" && echo -ne "${TARGET}${D}" >> "${CSV}"
    echo "     Subject: ${SUBJECT}" && echo -ne "${SUBJECT}${D}" >> "${CSV}"
    echo "SerialNumber: ${SERIAL_NUMBER}" && echo -ne "${SERIAL_NUMBER}${D}" >> "${CSV}"
    echo "   StartDate: ${START_DATE}" && echo -ne "${START_DATE}${D}" >> "${CSV}"
    echo "     EndDate: ${END_DATE}" && echo -ne "${END_DATE}${D}" >> "${CSV}"
    echo " Cert Thumbprint: ${CERT_THUMBPRINT}"  && echo -ne "${CERT_THUMBPRINT}${D}" >> "${CSV}"
    echo " SPKI Thumbprint: ${SPKI_THUMBPRINT}" && echo -ne "${SPKI_THUMBPRINT}" >> "${CSV}"
    echo "" >> "${CSV}"

done

In [ ]:
%%bash

FOLDER="data/badssl"

rm -rf ${FOLDER}